<a href="https://colab.research.google.com/github/davis689/binder/blob/master/Keq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sympy import *
from scipy.optimize import brentq,fsolve
import matplotlib.pyplot as plt
import requests
import lxml.html as lh
import pandas as pd
from functools import reduce


Set up

In [2]:
T,V,A,B,p,x,xi=var('T, V,A,B,p,x,xi')
R=8.3144598 

Import data from webbook.nist.gov for each molecule

In [3]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C10024972&Units=SI&Mask=1#Thermo-Gas' #N2O
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So=[]
So.append(float(df.iloc[2,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef=[]
coef.append([float(tmp[i]) for i in range(len(tmp))])

In [4]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C7782447&Units=SI&Mask=1#Thermo-Gas' #O2
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So.append(float(df.iloc[1,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef.append([float(tmp[i]) for i in range(len(tmp))])

In [5]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C10102440&Units=SI&Mask=1#Thermo-Gas' #NO2
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So.append(float(df.iloc[2,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef.append([float(tmp[i]) for i in range(len(tmp))])

Coefficients and the powers in the Shomate equation.

In [6]:
stoic=[-2,-3,4] # stoichiometry with reactant negative.
species=['N2O','O2','NO2']

In [7]:
#@title Input the temperature and pressure at which you want to calculate entropy change
pi = 1
pf =  .5#@param {type:"number"}
Ti = 298
Tf =  400#@param {type:"number"}
n_N2O =  1.5#@param {type:"number"}
n_O2 =  1.5#@param {type:"number"}
n_NO2 =  1.5#@param {type:"number"}
nrxn=[n_N2O,n_O2,n_NO2]

In [8]:
Hf=[sum([coef[i][j]/(j+1)*(T/1000)**(j+1) for j in range(4)]) for i in range(len(stoic))]
Hf=[Hf[i]-coef[i][4]*(1000/T)+coef[i][5]-coef[i][7] for i in range(len(stoic))] # temperature change for each species
DH=sum([Hf[i]*stoic[i] for i in range(len(stoic))])# total change from 298K added up using stoichiometry for each species.

Hfo=sum([coef[i][7]*stoic[i] for i in range(len(stoic))]) # standard reaction enthalpy change. The 7th element of coeff is the standard enthalpy.

DHtot=DH+Hfo #add temperature change enthalpy to standard enthalpy

print('standard enthalpy=',round(Hfo,2),'kJ/mol       enthalpy at',Tf,'K=',DHtot.subs(T,Tf).round(2),'kJ/mol') 

standard enthalpy= -31.72 kJ/mol       enthalpy at 400 K= -33.38 kJ/mol


In [9]:
S=[sum([coef[i][j]/(j)*(T/1000)**j for j in range(4) if j>0]) for i in range(len(stoic))] # take care of the polynomial part of the equation
S=[S[i]+coef[i][0]*ln(T/1000)-coef[i][4]/(2*(T/1000)**2)+coef[i][6] for i in range(len(stoic))] # add terms for the non-polynomial terms
DStot=sum([S[i]*stoic[i] for i in range(len(stoic))]) # add up the entropies for all species with the appropriate stoichiometry
DStot_p=DStot+R*ln(pf) #If we account for pressure here. Below for pressure accounted for in the delta G equation.
print('total change in entropy at',Tf,'K=',DStot.subs(T,Tf).round(2),'J/mol K            \ntotal change in entropy at',Tf,'K and',pf,'bar =',DStot_p.subs(T,Tf).round(2),'J/mol K')
print('Delta G from Delta H and Delta S',(DHtot*1000-Tf*DStot_p).subs(T,Tf).round(2),'J/mol') # calc DS for reaction and temperature change, for rxn, T, and p, and DG for same

total change in entropy at 400 K= -100.05 J/mol K            
total change in entropy at 400 K and 0.5 bar = -105.82 J/mol K
Delta G from Delta H and Delta S 8942.17 J/mol


In [10]:
[coef[i][7]+Hf[i].subs(T,Tf)-450*S[i].subs(T,Tf).round(2)/1000 for i in range(len(stoic))] #delta Gs for each species but using S not Delta S. This isn't right but it works if you take products minus reactants

[-18.1569286250167, -93.2155357607917, -76.0805916490833]

In [11]:
print('Standard entropyies: ',So) #standard entropies
print('Standard entropy change at T=',Tf,'K:', sum([So[i]*stoic[i] for i in range(len(stoic))]),'J/mol K') #standard entropy change at 298 K and 1 bar.

Standard entropyies:  [219.96, 205.15, 240.04]
Standard entropy change at T= 400 K: -95.21000000000015 J/mol K


In [12]:
[print(S[i].subs(T,Tf).round(2),'J/mol K') for i in range(len(stoic))]# entropy of each species at Tf
[print(Hf[i].subs(T,Tf).round(2),'kJ/mol') for i in range(len(stoic))] #enthapy change for each species to get to Tf
[print((Hf[i]+coef[i][7]).subs(T,Tf).round(2),'kJ/mol') for i in range(len(stoic))] #enthalpy of formation at Tf

231.90 J/mol K
213.87 J/mol K
251.34 J/mol K
4.15 kJ/mol
3.03 kJ/mol
3.93 kJ/mol
86.20 kJ/mol
3.03 kJ/mol
37.02 kJ/mol


[None, None, None]

In [13]:
DG=DHtot*1000-T*DStot+sum(stoic)*R*T*log(pf)
print('Delta G at Tf=',Tf,'K=',(DHtot*1000-T*DStot).subs(T,Tf).round(2),'J/mol         \nDelta G of pressure change=', (sum(stoic)*R*T*log(pf)).subs(T,Tf).round(2),'J/mol               \ntotal Delta G=',DG.subs(T,Tf).n().round(2),'J/mol') # DG... same as above, calculated differently

Delta G at Tf= 400 K= 6636.91 J/mol         
Delta G of pressure change= 2305.26 J/mol               
total Delta G= 8942.17 J/mol


In [14]:
Keq=exp(-DG/R/T)
print('Keq=',Keq.subs(T,Tf)) #calculate Keq

Keq= 0.0679665679167046


In [15]:
ch=[nrxn[i]+stoic[i]*xi for i in range(len(stoic))]; sm=sum(ch) #calculate moles with xi and sum them up.
num=[(ch[i]/sm)**stoic[i] for i in range(len(stoic)) if stoic[i]>0] #the numerator in Keq
den=[(ch[i]/sm)**stoic[i] for i in range(len(stoic)) if stoic[i]<0] #the denominator in Keq. Both are not needed except that the Keq solver works better if we don't use a complicated fraction.

In [16]:
ximin=max([-nrxn[i]/stoic[i] for i in range(len(stoic)) if stoic[i]>0]) #calculate the high end of the xi range
ximax=min([-nrxn[i]/stoic[i] for i in range(len(stoic)) if stoic[i]<0]) #calculate the low end of the xi range

print('xi is between',ximin,'and',ximax)

xi is between -0.375 and 0.5


Now we define a function that can be solved for $\xi$. The function needs to be equal to zero so we'll set it up so that $\frac{\Pi_i [products]_i^{a_i}}{\Pi_j[reactants]_j^{a_j}}-K_{eq}=0$, where $a$ is the stoichiometric coefficient for species $i$ or $j$. But really it's easier to solve if we take away the fraction and write it as $\Pi_i[products]_i^{a_i}-K_{eq}\times[reactants]_j^{a_j}=0$.

Because of this form, it is conventient to deal with the numerator and the denominator of the equilibrium constant separately. Each of these is just a product of each of the products or each of the reactants all to their appropriate powers. There is no $product$ function in most now current python versions but 'reduce(lambda x,y:x*y,list)' accomplishes this task for the lists of terms in the numerator ($num$) and the denominator ($den$). 

In [17]:
def K(x):
  num=[(ch[i]/sm).subs(xi,x)**stoic[i] for i in range(len(stoic)) if stoic[i]>0]
  den=[(ch[i]/sm).subs(xi,x)**-stoic[i] for i in range(len(stoic)) if stoic[i]<0]
  return reduce(lambda x,y:x*y,num)-Keq.subs(T,Tf)*reduce(lambda x,y:x*y,den) # 

In [18]:
#xi=np.linspace(ximin,ximax,100)


Now we use one of the equation solvers, here $brentq$, to solve our function between $\xi_{min}$ and $\xi_{max}$.

In [19]:
x=brentq(K,ximin,ximax) #find zeros of the function above between the minimum and maximum possible xi.
x

-0.1746707039102704

Now we can use $\xi$ to calculate the number of moles of each reactant and product at equilibrium.

In [20]:
n=[ch[i].subs(xi,x) for i in range(len(stoic))] # substitute xi into moles expressions to get answer.
output=[print(species[i],':',n[i].round(2),"mol") for i in range(len(stoic))]

N2O : 1.85 mol
O2 : 2.02 mol
NO2 : 0.80 mol


A test for consistency is always good. A calculation of the difference in $K_{eq}$ using moles and using $\Delta G$ should give zero. Here we allow it to be as big as $\pm10^{-8}$. 

In [21]:
kk=[(n[i]/sm.subs(xi,x))**stoic[i] for i in range(len(stoic))] #test equilibrium constant in terms of xi
print(abs(reduce(lambda x,y:x*y,kk)-Keq.subs(T,Tf))<10**-8) #make sure the K calculated from moles matches the K calculated from Delta G (or close enough). True means we're good.

True
